In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import random
import time
import math
import matplotlib.pyplot as plt
from IPython.core.debugger import Tracer; 
import itertools

In [2]:
random.seed(1)

In [3]:
almNum = 3
p_caus = 0.75
pCausInit = 0.9
ocorr = 100
t_amos = 60
hora_base = dt.datetime(2018, 1,9,9,0,0)

b = []
c = []

In [4]:
hora_base = dt.datetime(2018, 1,9,9,0,0)

In [5]:

hora_base = dt.datetime(2018, 1,9,9,0,0)
numOcorr = 0

a = np.zeros(1, dtype=int)
ocorr = 10
duracao = 60 #120sec
inicio = hora_base
tempoOcorA = []
tempoOcorB = []
tempoOcorC= []

b = []
b = np.zeros(1, dtype=int)
pAb = 1
inicioB = hora_base
atraso = 15 #seconds
duracaoB = 60


c = []
c = np.zeros(1, dtype=int)
pAc = 1
inicioC = hora_base
atraso = 15 #seconds
duracaoC = 120
count = 0;

index_fim_b = 0 
index_init_b =0
index_fim_a =0
index_init_a = 0
index_fim_c = 0
index_init_c = 0

while(numOcorr < ocorr):    
    hora_sort = random.normalvariate(3, 1)
    hora_ocor_inicio = inicio + dt.timedelta(hours=hora_sort)
    hora_ocor_fim = hora_ocor_inicio + dt.timedelta(seconds = duracao)

    #print('hora inicial', hora_ocor_inicio)
    #print('hora final', hora_ocor_fim)

    index_init_a = int(math.ceil((hora_ocor_inicio - hora_base).total_seconds()/t_amos))
    index_fim_a = int(math.ceil((hora_ocor_fim - hora_base).total_seconds()/t_amos))

    #print(index_init_a, index_fim_a)
    tempoOcorA.append([hora_ocor_inicio, hora_ocor_fim])

    p = random.uniform(0,1)
    #Cálculo de B
    if p <= pAb:
        hora_inicio_b = inicioB + dt.timedelta(seconds = ((index_init_a+1)*t_amos + atraso))
        hora_fim_b = hora_inicio_b + dt.timedelta(seconds=duracao)
        tempoOcorB.append([hora_inicio_b, hora_fim_b])  

        index_init_b = int(math.ceil((hora_inicio_b - inicioB).total_seconds()/t_amos))
        index_fim_b = int(math.ceil((hora_fim_b - inicioB).total_seconds()/t_amos))

        if(index_fim_b > b.size):
                b.resize(index_fim_b)

        for j in np.arange(index_init_b, index_fim_b +1):
            b[j-1] =  1    
    #Fim de B

    #Cálculo de C
    if p <= pAc:
        hora_inicio_c = inicioC + dt.timedelta(seconds = ((index_init_a+1)*t_amos + atraso))
        hora_fim_c = hora_inicio_c + dt.timedelta(seconds=duracao)
        tempoOcorC.append([hora_inicio_c, hora_fim_c])  

        index_init_c = int(math.ceil((hora_inicio_c - inicioC).total_seconds()/t_amos))
        index_fim_c = int(math.ceil((hora_fim_c - inicioC).total_seconds()/t_amos))
        if(index_fim_c > c.size):
            c.resize(index_fim_c)

    for j in np.arange(index_init_c, index_fim_c +1):
        c[j-1] = 1    
    #Fim de C

    if(index_fim_a < index_fim_b and  index_fim_b >= index_fim_c and index_fim_b > a.size):
        a.resize(index_fim_b)
    elif(index_fim_a> a.size or index_fim_a == index_fim_b and index_fim_b == c):
        a.resize(index_fim_a)
    elif(index_fim_a < index_fim_b and  index_fim_b <= index_fim_c and index_fim_c > a.size):
        a.resize(index_fim_c)

    for i in np.arange(index_init_a, index_fim_a + 1):
        a[i-1] = 1

    inicio = hora_ocor_inicio
    numOcorr = numOcorr + 1

sum(a)

20

In [26]:
sum(b)

20

In [6]:
a = a.astype(int)
b = b.astype(int)

In [27]:
k = 1 #horizonte de tempo de A
l = 1 #horizonte de tempo de B
h = 1

#larmes A, B
sizeSeries = a.size
transEntropy = 0
numStates = 2**(k + l  + 1)
combinations = list(map(list, itertools.product([0, 1], repeat=k+l+1)))
counting = np.zeros(numStates)
prob_cnjt = np.zeros(numStates)
a_prob_ind = []
b_prob_ind = []

#calcuo da probabilidade conjunta p(i_sub_t+1), i_sub_t**k, j_sub_t**l)
inicio = np.max([k,l]) - 1
for i in np.arange(inicio, sizeSeries - 1):
    for hk in np.arange(0,k):
            b_prob_ind.append(b[i - hk])
    for hl in np.arange(0,l):
            a_prob_ind.append(a[i - hl])
            
    ab = [a[i + h]] + a_prob_ind + b_prob_ind
    index_comb = combinations.index(ab)
    counting[index_comb] = counting[index_comb] + 1
    
    a_prob_ind = []
    b_prob_ind = []

total = sum(counting)
for i, cnt in enumerate(counting):
    prob_cnjt[i] = cnt/total        

In [28]:
prob_cnjt

array([ 0.97612086,  0.00925926,  0.00487329,  0.        ,  0.00487329,
        0.        ,  0.00487329,  0.        ])

In [29]:
#Cálculo da probabilidade marginal i_sub_t**
p_marg_i_k = np.zeros(2**k)
estados_i = list(map(list, itertools.product([0, 1], repeat=k)))
for i, est_i in enumerate(estados_i):
    for j, comb in enumerate(combinations):
        if  comb[1:k+1] == est_i:
            p_marg_i_k[i] = p_marg_i_k[i] + prob_cnjt[j]
    



In [30]:
sum(p_marg_i_k)

1.0

In [31]:
#Cálculo da probabilade conjunta p(i_t+h, i_t**k)
estados_ith_ik = list(map(list, itertools.product([0, 1], repeat=k + 1)))
p_cjnt_ith_ik = np.zeros(2**(k+1))
                         
for i, estado in enumerate(estados_ith_ik):
    for j, comb in enumerate(combinations):
        if comb[0:k+1] == estado:
            p_cjnt_ith_ik[i] = p_cjnt_ith_ik[i] + prob_cnjt[j]
            

In [32]:
sum(p_cjnt_ith_ik)

1.0

In [33]:
#Cálculo da conjunta p(ik, jl)
prob_cnjt_ik_jl = []
size = int(prob_cnjt.size/2)

for i in np.arange(0,size):
    prob_cnjt_ik_jl.append(prob_cnjt[i] + prob_cnjt[2**(k+l) + i])
prob_cnjt_ik_jl = np.asarray(prob_cnjt_ik_jl)

In [34]:
sum(prob_cnjt_ik_jl)

1.0

In [66]:
estados_i_j = list(map(list, itertools.product([0, 1], repeat=k+l)))

size = int(prob_cnjt.size/2)
num = np.zeros(2**(k+l+1))
count_zero  = 0
count_um = 0


for i in np.arange(0,size):
    for est in (estados_i_j):
        for j, comb in enumerate(combinations):
            #print(est)
            if comb[1:k+l+ 1] == est:
               # print(comb[1:k+l+1])
                if(comb[0] == 1):
                    count_um = count_um + prob_cnjt[j]
                else:
                    count_zero = count_zero + prob_cnjt[j]
    num[i] = count_zero/(count_zero+ count_um)
    num[i + 2**(k+l)] = count_um/(count_zero+ count_um)

In [36]:
sum(num)

3.9999999999999996

In [37]:
#Cálculo da probabilidade condicional do denominador
condicional_den = []

estados_i = list(map(list, itertools.product([0, 1], repeat=k)))
for i,est in enumerate(estados_ith_ik):
    index_i_marg = estados_i.index(est[1:k+1])
    condicional_den.append(p_cjnt_ith_ik[i]/p_marg_i_k[index_i_marg])

In [61]:
size = int((2**(k+1))/2)
count_zero  = 0
count_um = 0
den = np.zeros(2**(k+1))
print(size)
for i in np.arange(0, size):
    for est in estados_i:
        for j, comb in enumerate(estados_i_j):
            if(comb[1:k+1] == est):
                print(comb)
                if(comb[0] == 1):
                    count_um = count_um + p_cjnt_ith_ik[j]
                else:
                    count_zero = count_zero + p_cjnt_ith_ik[j]
    den[i] = count_zero/(count_zero+ count_um)
    den[i + 2**(k)] = count_um/(count_zero+ count_um)

2
[0, 0]
[1, 0]
[0, 1]
[1, 1]
[0, 0]
[1, 0]
[0, 1]
[1, 1]


In [79]:
condicional_den

[0.99507874015748032, 0.5, 0.0049212598425196841, 0.5]

In [71]:
den

array([ 0.99025341,  0.99025341,  0.00974659,  0.00974659])

In [72]:
num

array([ 0.99025341,  0.99025341,  0.99025341,  0.99025341,  0.00974659,
        0.00974659,  0.00974659,  0.00974659])

In [87]:
divisao_condicionais = np.zeros(num.size)
estados_comb_den = list(map(list, itertools.product([0, 1], repeat=1+k)))
for j, comb in enumerate(combinations):
    print(comb)
    divisao_condicionais[j] = num[j]/condicional_den[estados_comb_den.index(comb[0:k+1])]
    #print(num[j], den[estados_comb_den.index(comb[0:k+1])])
    print(estados_comb_den[estados_comb_den.index(comb[0:k+1])])
            
    

[0, 0, 0]
[0, 0]
[0, 0, 1]
[0, 0]
[0, 1, 0]
[0, 1]
[0, 1, 1]
[0, 1]
[1, 0, 0]
[1, 0]
[1, 0, 1]
[1, 0]
[1, 1, 0]
[1, 1]
[1, 1, 1]
[1, 1]


In [82]:
divisao_condicionais

array([ 0.99515081,  0.99515081,  1.98050682,  1.98050682,  1.98050682,
        1.98050682,  0.01949318,  0.01949318])

In [85]:
#cálculo da transentropia

#Calculo do log2 da divisao p(i_sub_t+h|i_sub_t**k, j_sub_t**l) /p(i_sub_t+h|i_t**k)
log_div_cond = np.log2(divisao_condicionais)
te = np.sum(prob_cnjt*log_div_cond)

In [86]:
te

-0.024986163763457428

In [25]:
# #Cálculo da probabilidade condicional do numerador

# #Shift circular a esquerda
# combinations_shifted = list(np.roll(combinations, -1))

# #calculo da probabilidade condicional p(i_sub_t+h|i_sub_t**k, j_sub_t**l) 
# condicional_num  = np.zeros(prob_cnjt.size)
# estados_i_j = list(map(list, itertools.product([0, 1], repeat=k+l)))
# for i, comb in enumerate(combinations_shifted):
#     index_prob_cnjt = combinations.index(list(comb))
#     index_est_i_j = estados_i_j.index(list(comb[0:k+l]))
#     if(prob_cnjt_ik_jl[index_est_i_j] == 0):
#         condicional_num[index_prob_cnjt] = (0)
#     else:
#         condicional_num[index_prob_cnjt] = (prob_cnjt[index_prob_cnjt]/prob_cnjt_ik_jl[index_est_i_j])
#     print("cnjt geral", prob_cnjt[index_prob_cnjt])
#     print('cjnt i e j', prob_cnjt_ik_jl[index_est_i_j])
# condicional_num = np.asarray(condicional_num)